#### Importações

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from ctgan import CTGAN
import numpy as np

ValueError: All ufuncs must have type `numpy.ufunc`. Received (<ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>, <ufunc 'sph_legendre_p'>)

#### Carrega e pré processa os dados

In [3]:
# Column names for the dataset
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Load the Adult dataset
train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# Load training data
train_data = pd.read_csv(train_url, header=None, names=columns, na_values=["?"], skipinitialspace=True)

# Load test data
test_data = pd.read_csv(test_url, header=None, names=columns, na_values=["?"], skipinitialspace=True, skiprows=1)
test_data["income"] = test_data["income"].str.strip(".")  # Clean income column in test set

# Remove the 'fnlwgt' column
train_data = train_data.drop(columns=['fnlwgt', 'education-num'])
test_data = test_data.drop(columns=['fnlwgt', 'education-num'])

In [1]:
train_data['capital-gain'] = train_data['capital-gain'].astype(int)
test_data['capital-gain'] = test_data['capital-gain'].astype(int)

NameError: name 'train_data' is not defined

In [1]:
train_data = train_data[train_data['capital-gain'] != 99999]
test_data = test_data[test_data['capital-gain'] != 99999]

NameError: name 'train_data' is not defined

#### Geração de dados sintéticos

In [6]:
discrete_columns = train_data.select_dtypes(include=["object"]).columns
ctgan = CTGAN(epochs=1)
ctgan.fit(train_data, discrete_columns)

: 

In [121]:
syndata = ctgan.sample(30000)

In [79]:
# Remove 1000 random samples from train_data
train_data = train_data.drop(train_data.sample(n=20000).index)

#### Transformações para modelagem

In [6]:
# Combine train and test data for consistent preprocessing
data = pd.concat([train_data, test_data], axis=0, ignore_index=True) #adicione e remova train_data e syndata para treinar ou nao com dados reais e sinteticos

# Combinar categorias da variável 'native-country'
data['native-country'] = data['native-country'].replace({
    country: 'Other' for country in data['native-country'].unique() if country != 'United-States'
})

# Encode the target variable
label_encoder = LabelEncoder()
data["income"] = label_encoder.fit_transform(data["income"])  # Encodes '<=50K' as 0 and '>50K' as 1


# Split features and target
X = data.drop(columns=["income"])
y = data["income"]

# Label Encoding para education, sex e race
for col in ["education", "sex", "native-country"]:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Define categorical and numerical columns
categorical_columns = X.select_dtypes(include=["object"]).columns
numerical_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_columns),  # Scale numerical features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns),  # One-hot encode categorical features
    ],
    remainder="passthrough"
)

# Split back into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(test_data), shuffle=False)

#### Pipeline para Regressão Logística

In [60]:
# Create a pipeline with preprocessing and classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000)),
])

#### Pipeline para Random Forest

In [62]:
# Create a pipeline with preprocessing and Random Forest classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
])

#### Treina o modelo

In [63]:
# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("ROC AUC:", roc_auc_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))


Accuracy: 0.8468153061851238

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.88      0.92      0.90     12435
        >50K       0.71      0.60      0.65      3846

    accuracy                           0.85     16281
   macro avg       0.79      0.76      0.78     16281
weighted avg       0.84      0.85      0.84     16281

ROC AUC: 0.7621508913432532
F1 Score: 0.6498174670036506


#### Hyperparameter tunning Random Forest

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [num_clients0, 200, 300, 500],
    "max_depth": [None, num_clients, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
}

# Create the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available processors
)

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Fit the model
random_search.fit(X_train_preprocessed, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

# Evaluate the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

#### Treina XGBoost

##### Usando XGBClassifier (mais alto nível)

In [78]:
# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Train the XGBoost model directly
model = XGBClassifier(eval_metric="logloss", n_jobs=-1)
model.fit(X_train_preprocessed, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_preprocessed)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test_preprocessed)[:, 1]))


Accuracy: 0.8724894048277133

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92     12435
        >50K       0.77      0.66      0.71      3846

    accuracy                           0.87     16281
   macro avg       0.83      0.80      0.81     16281
weighted avg       0.87      0.87      0.87     16281

ROC-AUC: 0.9256168477539263


##### Usando xgb.train (mais baixo nível)

In [79]:
import xgboost as xgb

In [80]:
train_dmatrix = xgb.DMatrix(data=X_train_preprocessed, label=y_train)

In [81]:
test_dmatrix = xgb.DMatrix(data=X_test_preprocessed, label=y_test)

In [82]:
model = xgb.train(
    params={"objective": "binary:logistic",  # Classificação binária
    "eval_metric": "auc",        # Métrica de avaliação
    "eta": 0.1,                      # Taxa de aprendizado (equivalente a learning_rate)
    "max_depth": 6,                   # Profundidade das árvores
    "nthread": -1},
    dtrain=train_dmatrix,
    num_boost_round=100,
    evals=[(test_dmatrix, "Test")]
)

[0]	Test-auc:0.88869
[1]	Test-auc:0.89441
[2]	Test-auc:0.90233
[3]	Test-auc:0.90325
[4]	Test-auc:0.90526
[5]	Test-auc:0.90534
[6]	Test-auc:0.90716
[7]	Test-auc:0.90772
[8]	Test-auc:0.90807
[9]	Test-auc:0.90868
[10]	Test-auc:0.90901
[11]	Test-auc:0.90956
[12]	Test-auc:0.91004
[13]	Test-auc:0.91041
[14]	Test-auc:0.91119
[15]	Test-auc:0.91184
[16]	Test-auc:0.91238
[17]	Test-auc:0.91295
[18]	Test-auc:0.91325
[19]	Test-auc:0.91369
[20]	Test-auc:0.91415
[21]	Test-auc:0.91436
[22]	Test-auc:0.91503
[23]	Test-auc:0.91538
[24]	Test-auc:0.91570
[25]	Test-auc:0.91599
[26]	Test-auc:0.91664
[27]	Test-auc:0.91665
[28]	Test-auc:0.91709
[29]	Test-auc:0.91753
[30]	Test-auc:0.91759
[31]	Test-auc:0.91860
[32]	Test-auc:0.91889
[33]	Test-auc:0.91918
[34]	Test-auc:0.91940
[35]	Test-auc:0.91957
[36]	Test-auc:0.92032
[37]	Test-auc:0.92037
[38]	Test-auc:0.92051
[39]	Test-auc:0.92055
[40]	Test-auc:0.92063
[41]	Test-auc:0.92087
[42]	Test-auc:0.92103
[43]	Test-auc:0.92156
[44]	Test-auc:0.92180
[45]	Test-auc:0.9220

In [83]:
y_pred = model.predict(test_dmatrix)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

AUC sem usar funcao do sklear

In [84]:
eval_results = model.eval_set(
            evals=[(test_dmatrix, "valid")],
            iteration=model.num_boosted_rounds() - 1,
        )
auc = round(float(eval_results.split("\t")[1].split(":")[1]), 4)

In [85]:
auc

0.9274

AUC com sklearn

In [86]:
roc_auc_score(y_test, y_pred)

0.9274362305413004

In [87]:
print("Accuracy:", accuracy_score(y_test, y_pred_binary))
print("\nClassification Report:\n", classification_report(y_test, y_pred_binary, target_names=label_encoder.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("f1_score_macro:", f1_score(y_test, y_pred_binary, average='macro'))
print("f1_score_micro:", f1_score(y_test, y_pred_binary, average='micro'))
print("f1_score_weighted:", f1_score(y_test, y_pred_binary, average='weighted'))
print("f1_score_none:", f1_score(y_test, y_pred_binary, average=None))

Accuracy: 0.8746391499293655

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.89      0.95      0.92     12435
        >50K       0.79      0.64      0.71      3846

    accuracy                           0.87     16281
   macro avg       0.84      0.79      0.81     16281
weighted avg       0.87      0.87      0.87     16281

ROC-AUC: 0.9274362305413004
f1_score_macro: 0.8136916070144624
f1_score_micro: 0.8746391499293655
f1_score_weighted: 0.869907075882171
f1_score_none: [0.92025163 0.70713158]


#### Hyperparameter tunning XGBoost

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.2, 0.5],
    "reg_alpha": [0, 0.01, 0.1, 1],
    "reg_lambda": [0.1, 1, 10]
}

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

class SklearnXGBClassifier(XGBClassifier):
    def __sklearn_tags__(self):
        return {}

# Create the model
xgb = SklearnXGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all processors
)


# Fit RandomizedSearchCV on training data
random_search.fit(X_train_preprocessed, y_train)

# Best parameters and model evaluation
print("Best Parameters:", random_search.best_params_)

# Make predictions
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)

# Evaluate the performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


#### Federated

In [35]:
from flwr_datasets.partitioner import IidPartitioner, DirichletPartitioner
from flwr_datasets import FederatedDataset
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

In [115]:
num_clients = 4

In [84]:
#Se for particionar de maneira IID
partitioner = IidPartitioner(num_partitions=num_clients)

In [129]:
#Se for particionar de maneira Nao IID
partitioner = DirichletPartitioner(
                num_partitions=num_clients,
                partition_by="income",
                alpha=0.1,
                min_partition_size=30,
                self_balancing=False
                )

In [130]:
# Create a FederatedDataset
fds = FederatedDataset(
            dataset="scikit-learn/adult-census-income",
            partitioners={"train": partitioner},
        )

In [131]:
# Load the partitions
partitions = [fds.load_partition(i, split="train") for i in range(num_clients)]

In [132]:
# Split each partition into train and test sets
splits = [partition.train_test_split(test_size=0.2, seed=42) for partition in partitions]

# Separate train and test splits into lists
train_partitions = [split["train"] for split in splits]
test_partitions = [split["test"] for split in splits]


In [133]:
# Convert each partition to a pandas DataFrame
train_partitions_df = [partition.to_pandas() for partition in train_partitions]
test_partitions_df = [partition.to_pandas() for partition in test_partitions]

In [134]:
# Combine train and test dataframes, para que o encoder capte todas categorias
combined_data_dfs = [pd.concat([train, test]) for train, test in zip(train_partitions_df, test_partitions_df)]

In [ ]:
#
for df in combined_data_dfs:
    df['capital_gain'] = df['capital-gain'].astype(int)
    df = df[df['capital-gain'] != 99999]

In [135]:
categorical_cols = combined_data_dfs[0].select_dtypes(include=["object"]).columns

In [ ]:
# Gera dados sinteticos para cada particao
ctgan = CTGAN(epochs=1)
syndata = []
for data in train_partitions_df:
    ctgan.fit(data, categorical_cols)
    syndata.append(ctgan.sample(1000))

In [ ]:
# Remove random samples from train_data e adiciona sintetico
for i, df in enumerate(train_partitions_df):
    df.drop(df.sample(n=1000).index)
    df = pd.concat([df, syndata[i]])

In [136]:
ordinal_encoder = OrdinalEncoder()

In [137]:
# Encode categorical columns em cada particao de treino e teste
for i in range(num_clients):
    combined_data_dfs[i][categorical_cols] = ordinal_encoder.fit_transform(combined_data_dfs[i][categorical_cols])
    train_partitions_df[i][categorical_cols] = ordinal_encoder.transform(train_partitions_df[i][categorical_cols])
    test_partitions_df[i][categorical_cols] = ordinal_encoder.transform(test_partitions_df[i][categorical_cols])

In [138]:
# Create DMatrixes for each partition
train_dmatrixes = []
test_dmatrixes = []

for partition in train_partitions_df:
    X_train = partition.drop(columns=["income"]).values
    y_train = partition["income"].values
    train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
    train_dmatrixes.append(train_dmatrix)

for partition in test_partitions_df:
    X_test = partition.drop(columns=["income"]).values
    y_test = partition["income"].values
    test_dmatrix = xgb.DMatrix(data=X_test, label=y_test)
    test_dmatrixes.append(test_dmatrix)

In [148]:
# Load saved model
with open("global_model_round1.json", "r") as f:  # Change round number as needed
    model_str = f.read()
    
# Convert string back to bytes
model_bytes = model_str.encode("utf-8")

# Define XGBoost parameters as dictionary
xgb_params = {
    'objective': 'binary:logistic',
    'eta': 0.1,
    'max_depth': 8,
    'eval_metric': 'auc',
    'nthread': 16,
    'num_parallel_tree': 1,
    'subsample': 1,
    'tree_method': 'hist'
}

# Create new booster and load model
bst = xgb.Booster(params=xgb_params)
bst.load_model(bytearray(model_bytes))

In [140]:
# Calcula metricas para cada particao
total_weighted_acc = 0
total_weighted_f1 = 0
len_test_partitions = 0
for test_dmatrix in test_dmatrixes:
    eval_results = bst.eval_set(
        evals=[(test_dmatrix, "valid")],
        iteration=bst.num_boosted_rounds() - 1,
    )
    auc = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
    print(f"AUC: {auc}")
    acc = accuracy_score(
                test_dmatrix.get_label(),
                bst.predict(test_dmatrix) > 0.5,
            )
    print(f"Accuracy: {acc}")
    f1 = f1_score(
                test_dmatrix.get_label(),
                bst.predict(test_dmatrix) > 0.5,
                average="macro"
            )
    print(f"F1 Score: {f1}")
    total_weighted_acc += acc * len(test_dmatrix.get_label())
    total_weighted_f1 += f1 * len(test_dmatrix.get_label())
    print(f"Partition weight: {len(test_dmatrix.get_label())}")
    len_test_partitions += len(test_dmatrix.get_label())

AUC: nan
Accuracy: 1.0
F1 Score: 1.0
Partition weight: 1059
AUC: 0.8942
Accuracy: 0.9983311081441922
F1 Score: 0.4995824285952898
Partition weight: 2996
AUC: nan
Accuracy: 1.0
F1 Score: 1.0
Partition weight: 1526
AUC: 0.8733
Accuracy: 0.962486602357985
F1 Score: 0.4904423812124522
Partition weight: 933


/Users/Mara/miniconda3/envs/XGBoostFed/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:12:41] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  warnings.warn(smsg, UserWarning)
/Users/Mara/miniconda3/envs/XGBoostFed/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [10:12:41] WARNING: /Users/runner/work/xgboost/xgboost/src/metric/auc.cc:324: Dataset is empty, or contains only positive or negative samples.
  warnings.warn(smsg, UserWarning)


In [141]:
#Calcula media das metricas de cada particao
avg_acc = total_weighted_acc / len_test_partitions
avg_f1 = total_weighted_f1 / len_test_partitions

print(f"Average Accuracy: {avg_acc}")
print(f"Average F1 Score: {avg_f1}")

Average Accuracy: 0.9938593797973595
Average F1 Score: 0.6968577982411278


In [147]:
for combined_data_df in combined_data_dfs:
    print(f"Class distribution: {combined_data_df['income'].value_counts()}")

Class distribution: income
0.0    5294
Name: count, dtype: int64
Class distribution: income
0.0    14946
1.0       30
Name: count, dtype: int64
Class distribution: income
0.0    7627
Name: count, dtype: int64
Class distribution: income
0.0    4480
1.0     184
Name: count, dtype: int64
